In [1]:
#import libraries
import lief
import os
from capstone import *
import re

In [2]:
# Find Rc4 algo 
def find_pattern(pattern,data):
    instruct = []
    string = ""
    algo_bytes = data[re.search(pattern,data).start():re.search(pattern,data).end()]
    md = Cs(CS_ARCH_X86, CS_MODE_32)
    md.detail = True
    
    for i in md.disasm(algo_bytes, 0):
        
        string = i.mnemonic  + ' ' + i.op_str
        instruct.append(string)
  
    return instruct 

In [3]:
def parse_binary(binary):
   pe = lief.parse(binary)
   section = pe.get_section(".text") 
   hex_data = ''.join(format(x, '02x') for x in section.content)
   transform = bytearray.fromhex(hex_data)
   
   return transform

In [4]:
def get_instructions(file):

    data = parse_binary(file)
    pattern = rb'(...)\x50(...)\x51(...)\x52\x68(..)\x00\x00(...)\x50\xff\x15(....)'
    pattern2 = rb'(...)\x50\x68\x11\x00\x28\x00\xff\x75\xfc\x68(..)\x00\x00(...)\xff\x15\x18(...)'

    if re.search(pattern,data):
        print('pattern 1 found')
        return find_pattern(pattern,data)

    elif re.search(pattern2,data):
        print('pattern 2 found')
        return find_pattern(pattern2,data)
        
    else:
        return rb'(\x90\x90\x90)'

In [5]:
def Jaccard_Similarity(x, y):

    intersection_cardinality = len(list(set(x).intersection(y)))
    union_cardinality = (len(x) + len(y)) - intersection_cardinality
    return float(intersection_cardinality) / union_cardinality

In [6]:
#Parse each file within the directory path 

MIN_SIMILARITY = 0.6
file_instructs = dict()
binaries = []

for file in os.listdir('Hancitor'):
    binaries.append(os.path.join('./Hancitor',file))

for bin in binaries:
    instruct = get_instructions(bin)
    file_instructs[bin] = instruct
    print(bin, '--->',instruct,'\n')


keys = list(file_instructs.keys())  
similarities = []
for i in range(len(keys)):
    for j in range(i+1, len(keys)):
        f1 = file_instructs[keys[i]]
        f2 = file_instructs[keys[j]]
        similarity = Jaccard_Similarity(f1,f2)
       
        print('Similarity between', keys[i], 'and', keys[j],'=', similarity)
        # Min similarity is set to 6
        if similarity < MIN_SIMILARITY:
            similarities.append((keys[i], keys[j], similarity))

pattern 1 found
./Hancitor/Hancitor3.bin ---> ['lea eax, [ebp - 0xc]', 'push eax', 'mov ecx, dword ptr [ebp - 0x14]', 'push ecx', 'mov edx, dword ptr [ebp - 4]', 'push edx', 'push 0x6801', 'mov eax, dword ptr [ebp - 8]', 'push eax', 'call dword ptr [0x584018]'] 

pattern 1 found
./Hancitor/Hancitor2.bin ---> ['lea eax, [ebp - 0xc]', 'push eax', 'mov ecx, dword ptr [ebp - 0x14]', 'push ecx', 'mov edx, dword ptr [ebp - 4]', 'push edx', 'push 0x6801', 'mov eax, dword ptr [ebp - 8]', 'push eax', 'call dword ptr [0x5c4018]'] 

pattern 1 found
./Hancitor/Hancitor8.bin ---> ['lea eax, [ebp - 0xc]', 'push eax', 'mov ecx, dword ptr [ebp - 0x14]', 'push ecx', 'mov edx, dword ptr [ebp - 4]', 'push edx', 'push 0x6801', 'mov eax, dword ptr [ebp - 8]', 'push eax', 'call dword ptr [0x5c4018]'] 

pattern 1 found
./Hancitor/Hancitor9.bin ---> ['lea eax, [ebp - 0xc]', 'push eax', 'mov ecx, dword ptr [ebp - 0x14]', 'push ecx', 'mov edx, dword ptr [ebp - 4]', 'push edx', 'push 0x6801', 'mov eax, dword ptr

In [7]:
print()
print('%20s %25s %30s' % ('Binary_1', 'Binary_2', 'Similarity'))
print('-' * 100)
for sim_item in similarities:
    print('%25s %25s %20.2f' % (sim_item[0][11:30], sim_item[1][11:30], sim_item[2]))


            Binary_1                  Binary_2                     Similarity
----------------------------------------------------------------------------------------------------
            Hancitor3.bin             Hancitor1.bin                 0.13
            Hancitor3.bin            Hancitor10.bin                 0.13
            Hancitor2.bin             Hancitor1.bin                 0.13
            Hancitor2.bin            Hancitor10.bin                 0.13
            Hancitor8.bin             Hancitor1.bin                 0.13
            Hancitor8.bin            Hancitor10.bin                 0.13
            Hancitor9.bin             Hancitor1.bin                 0.13
            Hancitor9.bin            Hancitor10.bin                 0.13
            Hancitor4.bin             Hancitor1.bin                 0.13
            Hancitor4.bin            Hancitor10.bin                 0.13
            Hancitor1.bin             Hancitor7.bin                 0.13
            Hanci